In [13]:
import pandas as pd
import numpy as np

In [165]:
def resample_to_constant_volume_original(df, volume_per_candle):
    # sort by time
    #df = df.sort_values(by='time')
    # create cumulative volume column
    df['cumulative_volume'] = df['volume'].cumsum()
    
    print(len(df))
    # initialize the new dataframe to hold resampled data
    resampled_df = pd.DataFrame(columns=['open', 'high', 'low', 'close','volume'])
    
    start_index = 0
    end_index = 0
    cumulative_volume = 0
    for i in range(len(df)):
        cumulative_volume += df.iloc[i]['volume']
        end_index = i
        if cumulative_volume >= volume_per_candle : # make function dynamic in next round
            candle_df = df.iloc[start_index:end_index+1]
            open_price = candle_df.iloc[0]['open']
            high_price = candle_df['high'].max()
            low_price = candle_df['low'].min()
            close_price = candle_df.iloc[-1]['close']
            date_end = df.iloc[start_index]["date"]

            resampled_df = resampled_df.append({'open': open_price, 'high': high_price, 'low': low_price, 
                                                'close': close_price, 'volume': cumulative_volume,
                                                 "date_end": date_end}, 
                                                 ignore_index=True)
            cumulative_volume = 0
            start_index = end_index + 1
    
    if start_index < len(df):
        candle_df = df.iloc[start_index:len(df)]
        open_price = candle_df.iloc[0]['open']
        high_price = candle_df['high'].max()
        low_price = candle_df['low'].min()
        close_price = candle_df.iloc[-1]['close']
        resampled_df = resampled_df.append({'open': open_price, 'high': high_price, 'low': low_price, 'close': close_price, 'volume': volume_per_candle}, ignore_index=True)

    return resampled_df

def resample_to_constant_volume(df, volume_per_candle, convertTime= False):
    # sort by time
    #df = df.sort_values(by='time')
    # create cumulative volume column
    df['cumulative_volume'] = df['volume'].cumsum()
    
    print(len(df))
    # initialize the new dataframe to hold resampled data
    resampled_df = pd.DataFrame(columns=['open', 'high', 'low', 'close','volume'])
    
    start_index = 0
    end_index = 0
    cumulative_volume = 0
    for i in range(len(df)):
        cumulative_volume += df.iloc[i]['volume']
        end_index = i
        if cumulative_volume >= volume_per_candle : # make function dynamic in next round
            candle_df = df.iloc[start_index:end_index+1]
            open_price = candle_df.iloc[0]['open']
            high_price = candle_df['high'].max()
            low_price = candle_df['low'].min()
            close_price = candle_df.iloc[-1]['close']
            date_end = df.iloc[start_index]["date"]
            #x =  pd.to_datetime(datetime.strptime(mask, '%Y-%m-%d %H:%M:%S'))
            ## similarly for date_end,, this is to quickly find what 5 minute candle it belong to
            #date_end = x.strftime(f"%Y-%m-%d %H:{5*int(x.minute/5)}:%S")
            if convertTime:
                mask = df.iloc[end_index]["date"].split("+")[0]
                x =  pd.to_datetime(datetime.strptime(mask, '%Y-%m-%d %H:%M:%S'))
                ## similarly for date_end,, this is to quickly find what 5 minute candle it belong to
                temp = ("0"+str(5*int(x.minute/5)))
                date_end = x.strftime(f"%Y-%m-%d %H:{temp[-2:]}:%S")
            """resampled_df = resampled_df.append({'open': open_price, 'high': high_price, 'low': low_price, 
                                                'close': close_price, 'volume': cumulative_volume,
                                                 "date_end": date_end}, 
                                                 ignore_index=True)"""
            for i in range(int(cumulative_volume/volume_per_candle)):
                resampled_df = resampled_df.append({'open': open_price, 'high': high_price, 'low': low_price, 
                                                    'close': close_price, 'volume': cumulative_volume,
                                                     "date_end": date_end}, 
                                                     ignore_index=True)
            cumulative_volume = 0
            start_index = end_index + 1
    
    if start_index < len(df):
        candle_df = df.iloc[start_index:len(df)]
        open_price = candle_df.iloc[0]['open']
        high_price = candle_df['high'].max()
        low_price = candle_df['low'].min()
        close_price = candle_df.iloc[-1]['close']
        volume = sum(candle_df["volume"])
        date_end = df.iloc[start_index]["date"]
        #x =  pd.to_datetime(datetime.strptime(mask, '%Y-%m-%d %H:%M:%S'))
        ## similarly for date_end,, this is to quickly find what 5 minute candle it belong to
        #date_end = x.strftime(f"%Y-%m-%d %H:{5*int(x.minute/5)}:%S")
        if convertTime:
            mask = df.iloc[end_index]["date"].split("+")[0]
            x =  pd.to_datetime(datetime.strptime(mask, '%Y-%m-%d %H:%M:%S'))
            ## similarly for date_end,, this is to quickly find what 5 minute candle it belong to
            temp = ("0"+str(5*int(x.minute/5)))
            date_end = x.strftime(f"%Y-%m-%d %H:{temp[-2:]}:%S")
        resampled_df = resampled_df.append({'open': open_price, 'high': high_price, 'low': low_price, 'close': close_price, 'volume': volume,"date_end": str(date_end)}, ignore_index=True)

    return resampled_df

In [15]:
df = pd.read_csv("ACC_5min-2.csv")

In [16]:
df.rename(columns = {"Interval" : "date"}, inplace = True)
df.drop(columns = df.columns[:3], inplace=True)
df.head()

date     open     high      low    close  volume
0  2018-01-01 09:15  1763.90  1763.90  1758.65  1758.65  3495.0
1  2018-01-01 09:20  1758.65  1761.00  1754.60  1754.80  3147.0
2  2018-01-01 09:25  1754.80  1756.35  1748.70  1750.15  4762.0
3  2018-01-01 09:30  1750.15  1751.25  1748.00  1749.40  2001.0
4  2018-01-01 09:35  1749.20  1751.15  1748.45  1749.25  2164.0

In [17]:
df[df["date"] == "2020-01-01 09:25"].index

Int64Index([33699], dtype='int64')

In [18]:
i = 39839
df["date"][i]

'2020-05-12 10:25'

In [21]:
# import warnings
# warnings.filterwarnings('ignore')
# df_vol_sampled = resample_to_constant_volume(df[33380:],volume_per_candle)

In [113]:
import plotly.graph_objs as go
import pandas as pd

def plot(df3):
    
    # Create the candlestick chart
    candlestick = go.Candlestick(x=df3.index, open=df3['open'], close=df3['close'], high=df3['high'], low=df3['low'])

    # Add a line plot with secondary x-axis
    line = go.Scatter(x=df3['date_end'], y=df3['close'], name='date_end', xaxis='x2')

    # Create the figure with both the candlestick and line plots
    fig = go.Figure(data=[candlestick,line])

    # Update the layout to add a secondary x-axis
    fig.update_layout(xaxis2=dict(title='date_end', overlaying='x', side='top'))
    fig.update_layout(xaxis_rangeslider_visible=False)

    # Show the chart
    fig.show()
    return

In [186]:
def is_body_size_greater_than_nx_average(dataframe, index,atr_multiple,look_back):
    body_sizes = (dataframe['close'] - dataframe['open']).iloc[index - look_back:index].abs()
    average_body_size = body_sizes.mean()
    current_body_size = abs(dataframe['close'].iloc[index] - dataframe['open'].iloc[index])
    return current_body_size > atr_multiple * average_body_size

In [187]:
df[df["date"] == "2020-05-12 10:25"]

date     open    high      low    close  volume
39839  2020-05-12 10:25  1157.45  1159.0  1155.25  1156.75  3462.0

In [196]:
def imbalance_from_volume(df, dateTime_index_to_check):
    
    ## df is 1min
    
    
    """" Takes volume sampled data and returns a list of imbalanced candlesticks """
    
    df = df.reset_index()
    ##index of the dateTime
    
    ind = df[df["date"] == dateTime_index_to_check].index[0]
    print(df)
    df = df[:ind+1]
    print(df)
    
    ##end  index to be the +5 min of dateTime index
    x =  pd.to_datetime(datetime.strptime(dateTime_index_to_check, '%Y-%m-%d %H:%M:%S'))
    endTime = x.strftime(f"%Y-%m-%d %H:{5*int((x.minute+5)/5)}:%S")
    end_index = df[df["date"] == dateTime_index_to_check].index[0] ## corner case 15:30
    
    #print(data["volume"].rolling(window=80000).mean()[ind])
    #vol_resampled_data = resample_to_constant_volume(df[ind-2000:end_index],df["volume"].rolling(window=80000).mean()[ind]*1.7)
    
    volume_per_candle = df["volume"].rolling(window=80000).quantile(0.95)[ind]*2.5
    vol_resampled_data1 = resample_to_constant_volume(df[ind-20000:end_index], volume_per_candle ,convertTime = True)
    

    
    index_in_vol = vol_resampled_data1.index[-1]#vol_resampled_data1[vol_resampled_data1["date_end"] == dateTime_index_to_check].index[0]

    if ((vol_resampled_data1[["close" , "open", "low"]] == vol_resampled_data1[["close" , "open", "low"]].loc[index_in_vol].shift()).all()
        & is_body_size_greater_than_nx_average(vol_resampled_data1, index_in_vol ,2,30)) :
        return True


    else :
        return False
        

    

    

In [197]:
x= imbalance_from_volume(df_1min.loc[66028:], "2022-10-10 13:15:00")

         index                 date    close     high      low     open  \
0        66028  2015-11-05 11:46:00  1390.50  1391.85  1390.35  1390.45   
1        66029  2015-11-05 11:47:00  1390.50  1390.50  1390.50  1390.50   
2        66030  2015-11-05 11:48:00  1391.00  1391.75  1390.50  1390.50   
3        66031  2015-11-05 11:49:00  1391.95  1391.95  1391.00  1391.00   
4        66032  2015-11-05 11:50:00  1392.00  1392.00  1391.95  1391.95   
...        ...                  ...      ...      ...      ...      ...   
594500  660528  2022-10-24 19:10:00  2255.00  2255.90  2254.55  2255.80   
594501  660529  2022-10-24 19:11:00  2255.95  2255.95  2254.55  2255.75   
594502  660530  2022-10-24 19:12:00  2256.25  2256.25  2255.35  2256.00   
594503  660531  2022-10-24 19:13:00  2255.00  2255.95  2254.55  2255.50   
594504  660532  2022-10-24 19:14:00  2256.00  2259.00  2254.00  2255.00   

        volume  
0          703  
1            0  
2           95  
3           26  
4            1

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
x

In [171]:
str(x["date_end"][5])

'2022-07-15 14:35:00'

In [78]:
if (vol_resampled_data1[["close" , "open", "low"]].iloc[3] == vol_resampled_data1[["close" , "open", "low"]]shift()).all() :
    print("BKL")

BKL


In [96]:
plot(x)

In [26]:
df_1min = pd.read_csv("ACC_minute_data_with_indicators.csv")
df_1min.head()

date   close     high     low    open  volume  \
0  2015-02-02 10:18:00+05:30  1544.0  1545.00  1543.1  1545.0     220   
1  2015-02-02 10:19:00+05:30  1543.7  1544.00  1543.0  1544.0      29   
2  2015-02-02 10:20:00+05:30  1543.0  1543.70  1542.5  1543.7      28   
3  2015-02-02 10:21:00+05:30  1542.3  1543.00  1542.3  1543.0     125   
4  2015-02-02 10:22:00+05:30  1542.1  1542.15  1541.1  1541.1     113   

      sma5     sma10        sma15      sma20  ...      fastd  fastksr  \
0  1544.94  1545.890  1546.756667  1546.9950  ...  31.307634      0.0   
1  1544.63  1545.545  1546.446667  1546.8650  ...  23.381708      0.0   
2  1544.22  1545.205  1546.013333  1546.5800  ...  15.620034      0.0   
3  1543.64  1544.635  1545.533333  1546.3950  ...   9.497585      0.0   
4  1543.02  1544.235  1545.143333  1546.0725  ...  13.377926      0.0   

        fastdsr     ULTOSC       WILLR       ATR  Trange     TYPPRICE  \
0  8.760123e+00  43.728004  -86.466165  2.056306     2.1  1544.033333   
1  4.380062e+00  47.390635  -89.629630  1.980855     1.0  1543.566667   
2  1.184238e-15  43.139523  -92.857143  1.925080     1.2  1543.066667   
3  1.184238e-15  44.657886 -100.000000  1.837574     0.7  1542.533333   
4  1.184238e-15  54.066603  -86.111111  1.792033     1.2  1541.783333   

   HT_DCPERIOD      BETA  
0    27.840336  0.138547  
1    27.115144  0.094738  
2    26.735549  0.336124  
3    26.601907  0.608042  
4    26.691996  0.253168  

[5 rows x 60 columns]

In [27]:
df_1min.drop(columns = df_1min.columns[6:], inplace=True)

In [28]:
from datetime import datetime
def ss(x):
    return datetime.strptime(x,"%Y-%m-%d %H:%M:%S+05:30").strftime("%Y-%m-%d %H:%M:%S")

In [29]:
df_1min["date"]=list(map(ss,df_1min["date"]))
df_1min

date    close     high      low     open  volume
0       2015-02-02 10:18:00  1544.00  1545.00  1543.10  1545.00     220
1       2015-02-02 10:19:00  1543.70  1544.00  1543.00  1544.00      29
2       2015-02-02 10:20:00  1543.00  1543.70  1542.50  1543.70      28
3       2015-02-02 10:21:00  1542.30  1543.00  1542.30  1543.00     125
4       2015-02-02 10:22:00  1542.10  1542.15  1541.10  1541.10     113
...                     ...      ...      ...      ...      ...     ...
660528  2022-10-24 19:10:00  2255.00  2255.90  2254.55  2255.80     199
660529  2022-10-24 19:11:00  2255.95  2255.95  2254.55  2255.75     227
660530  2022-10-24 19:12:00  2256.25  2256.25  2255.35  2256.00     594
660531  2022-10-24 19:13:00  2255.00  2255.95  2254.55  2255.50     267
660532  2022-10-24 19:14:00  2256.00  2259.00  2254.00  2255.00     365

[660533 rows x 6 columns]

In [57]:
plot(x)

In [32]:
x.iloc[935]

open                    2239.05
high                     2243.6
low                     2238.35
close                   2242.55
volume                     8247
date_end    2022-10-19 12:29:00
Name: 935, dtype: object

In [12]:
x.iloc[1730]

NameError: name 'x' is not defined

In [180]:
x.iloc[1760]

open                     2252.4
high                    2254.65
low                     2250.15
close                    2253.1
volume                     3888
date_end    2022-10-19 14:22:00
Name: 1760, dtype: object